# Présentation du projet - Rapport interactif

# I) Appel des ressources nécessaires - Modules 

In [6]:
import os
import numpy as np
from PIL import Image
import tensorflow.keras as keras

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# II) Constitution du dataset

In [2]:
CLASSES = ["espagne", "france", "japon"]
CLASSES_SIZE = len(CLASSES)

folder = "..\\dataset\\try"

test_file = os.path.join(folder, "test_file.png")

MAIN_DATASET_FOLDER = "..\\dataset\\"

TEST_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "test")
TRAIN_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "train")

TEST_ESPAGNE_FOLDER = os.path.join(TEST_SUBFOLDER, "espagne")
TEST_FRANCE_FOLDER = os.path.join(TEST_SUBFOLDER, "france")
TEST_JAPON_FOLDER = os.path.join(TEST_SUBFOLDER, "japon")

TRAIN_ESPAGNE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "espagne")
TRAIN_FRANCE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "france")
TRAIN_JAPON_FOLDER = os.path.join(TRAIN_SUBFOLDER, "japon")

def fill_x_and_y_with_images_and_labels(folder, classe, dataset):

    for file in os.listdir(folder):

        file_path = os.path.join(folder, file)
        image = Image.open(file_path)
        image = image.resize((32, 32))
        
        if image.mode in ("RGB", "P"): image = image.convert("RGBA")
            
        im_arr = np.array(image).flatten()
        im_arr = im_arr / 255.0

        completed_dataset = {}
        completed_dataset["value"] = im_arr
        completed_dataset["classe"] = classe

        dataset.append(completed_dataset.copy())


def import_dataset(folder, classe):

    dataset = []

    fill_x_and_y_with_images_and_labels(folder, classe, dataset)

    return dataset

x_test = []
y_test = []

x_train = []
y_train = []

classe_spain_expected_outputs = [1, 0, 0]
dataset_test_for_spain = import_dataset(TEST_ESPAGNE_FOLDER, classe_spain_expected_outputs)
dataset_train_for_spain = import_dataset(TRAIN_ESPAGNE_FOLDER, classe_spain_expected_outputs)

classe_france_expected_outputs = [0, 1, 0]
dataset_test_for_france = import_dataset(TEST_FRANCE_FOLDER, classe_france_expected_outputs)
dataset_train_for_france = import_dataset(TRAIN_FRANCE_FOLDER, classe_france_expected_outputs)

classe_japan_expected_outputs = [0, 0, 1]
dataset_test_for_japan = import_dataset(TEST_JAPON_FOLDER, classe_japan_expected_outputs)
dataset_train_for_japan = import_dataset(TRAIN_JAPON_FOLDER, classe_japan_expected_outputs)

final_dataset_test = dataset_test_for_spain + dataset_test_for_france + dataset_test_for_japan
final_dataset_train = dataset_train_for_spain + dataset_train_for_france + dataset_train_for_japan

for image_test in final_dataset_test:
    x_test.append(image_test["value"].tolist())
    y_test.append(image_test["classe"])

for image_train in final_dataset_train:
    x_train.append(image_train["value"].tolist())
    y_train.append(image_train["classe"])
    
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

# III) Implémentation des modèles

# III) A) Implémentation d'un perceptron simple

In [3]:
def train_mlp_wl_model():
    
    mlp_wl_model = keras.models.Sequential()
    mlp_wl_model.add(keras.layers.Input(shape=x_train.shape[1:]))
    mlp_wl_model.add(keras.layers.Dense(3, activation=keras.activations.softmax))

    mlp_wl_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss=keras.losses.CategoricalCrossentropy(from_logits=False), metrics=keras.metrics.CategoricalAccuracy(name="accuracy"))
    
    print(mlp_wl_model.summary())
    
    predicted_values_before_train = mlp_wl_model.predict(x_train)

    model_name = 'mlp_wl_model'
    log_dir_path = './logs'

    if not os.path.exists(log_dir_path):
        os.mkdir(log_dir_path)

    full_log_path = os.path.join(log_dir_path, model_name)

    if not os.path.exists(full_log_path):
        os.mkdir(full_log_path)

    mlp_wl_model.fit(x_train, y_train, epochs=1000, validation_data=(x_test, y_test), callbacks=[

        keras.callbacks.TensorBoard(full_log_path, )

    ])

    print("Valeurs de prédiction avant entrainement : \n", predicted_values_before_train)
    print("Valeurs de prédiction après entrainement : \n", mlp_wl_model.predict(x_train))
    
    mlp_wl_model.save("../saves/mlp_wl_model.h5")

In [4]:
def predict_mlp_wl_model():
    
    loaded_mlp_wl_model = keras.models.load_model("../saves/mlp_wl_model.h5")
    
    image = Image.open(test_file)
    if image.mode in ("RGB", "P"): image = image.convert("RGBA")

    image = image.resize((32, 32))

    im_arr = np.array(image).flatten()
    im_arr = im_arr / 255.0
    im_arr = im_arr.tolist()
    im_arr = np.array(im_arr)
    im_arr = np.expand_dims(im_arr, axis=0)
    
    predict = loaded_mlp_wl_model.predict(im_arr)
    predictions = predict.tolist()
    
    predicted_values = []
    
    for sublist in predictions:
        
        for item in sublist:
            predicted_values.append(item)

    tab_predict = []

    for i in range(3):
        tab_predict.append(predicted_values[i])

    max_index = tab_predict.index(max(tab_predict))

    print(tab_predict)
    print(CLASSES[max_index])

# III) B) Implémentation d'un PCM avec une couche cachée à 8 neurones

In [5]:
def train_mlp_ol_en_model():
    
    mlp_ol_en_model = keras.models.Sequential()
    mlp_ol_en_model.add(keras.layers.Input(shape=x_train.shape[1:]))
    mlp_ol_en_model.add(keras.layers.Dense(8, activation=keras.activations.relu))
    mlp_ol_en_model.add(keras.layers.Dense(3, activation=keras.activations.softmax))

    mlp_ol_en_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss=keras.losses.CategoricalCrossentropy(from_logits=False), metrics=keras.metrics.CategoricalAccuracy(name="accuracy"))
    
    print(mlp_ol_en_model.summary())
    
    predicted_values_before_train = mlp_ol_en_model.predict(x_train)

    model_name = 'mlp_ol_en_model'
    log_dir_path = './logs'

    if not os.path.exists(log_dir_path):
        os.mkdir(log_dir_path)

    full_log_path = os.path.join(log_dir_path, model_name)

    if not os.path.exists(full_log_path):
        os.mkdir(full_log_path)

    mlp_ol_en_model.fit(x_train, y_train, epochs=1000, validation_data=(x_test, y_test), callbacks=[

        keras.callbacks.TensorBoard(full_log_path, )

    ])

    print("Valeurs de prédiction avant entrainement : \n", predicted_values_before_train)
    print("Valeurs de prédiction après entrainement : \n", mlp_ol_en_model.predict(x_train))
    
    mlp_ol_en_model.save("../saves/mlp_ol_en_model.h5")

In [6]:
def predict_mlp_ol_en_model():
    
    loaded_mlp_ol_en_model = keras.models.load_model("../saves/mlp_ol_en_model.h5")
    
    image = Image.open(test_file)
    if image.mode in ("RGB", "P"): image = image.convert("RGBA")

    image = image.resize((32, 32))

    im_arr = np.array(image).flatten()
    im_arr = im_arr / 255.0
    im_arr = im_arr.tolist()
    im_arr = np.array(im_arr)
    im_arr = np.expand_dims(im_arr, axis=0)
    
    predict = loaded_mlp_ol_en_model.predict(im_arr)
    predictions = predict.tolist()
    
    predicted_values = []
    
    for sublist in predictions:
        
        for item in sublist:
            predicted_values.append(item)

    tab_predict = []

    for i in range(3):
        tab_predict.append(predicted_values[i])

    max_index = tab_predict.index(max(tab_predict))

    print("\n", tab_predict, "\n")
    print(CLASSES[max_index])

# III) C) Implémentation d'un PMC avec une couche cachée à 32 neurones

In [7]:
def train_mlp_ol_ttn_model():
    
    mlp_ol_ttn_model = keras.models.Sequential()
    mlp_ol_ttn_model.add(keras.layers.Input(shape=x_train.shape[1:]))
    mlp_ol_ttn_model.add(keras.layers.Dense(32, activation=keras.activations.relu))
    mlp_ol_ttn_model.add(keras.layers.Dense(3, activation=keras.activations.softmax))

    mlp_ol_ttn_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss=keras.losses.CategoricalCrossentropy(from_logits=False), metrics=keras.metrics.CategoricalAccuracy(name="accuracy"))
    
    print(mlp_ol_ttn_model.summary())
    
    predicted_values_before_train = mlp_ol_ttn_model.predict(x_train)

    model_name = 'mlp_ol_ttn_model'
    log_dir_path = './logs'

    if not os.path.exists(log_dir_path):
        os.mkdir(log_dir_path)

    full_log_path = os.path.join(log_dir_path, model_name)

    if not os.path.exists(full_log_path):
        os.mkdir(full_log_path)

    mlp_ol_ttn_model.fit(x_train, y_train, epochs=1000, validation_data=(x_test, y_test), callbacks=[

        keras.callbacks.TensorBoard(full_log_path, )

    ])

    print("Valeurs de prédiction avant entrainement : \n", predicted_values_before_train)
    print("Valeurs de prédiction après entrainement : \n", mlp_ol_ttn_model.predict(x_train))
    
    mlp_ol_ttn_model.save("../saves/mlp_ol_ttn_model.h5")

In [8]:
def predict_mlp_ol_ttn_model():
    
    loaded_mlp_ol_ttn_model = keras.models.load_model("../saves/mlp_ol_ttn_model.h5")
    
    image = Image.open(test_file)
    if image.mode in ("RGB", "P"): image = image.convert("RGBA")

    image = image.resize((32, 32))

    im_arr = np.array(image).flatten()
    im_arr = im_arr / 255.0
    im_arr = im_arr.tolist()
    im_arr = np.array(im_arr)
    im_arr = np.expand_dims(im_arr, axis=0)
    
    predict = loaded_mlp_ol_ttn_model.predict(im_arr)
    predictions = predict.tolist()
    
    predicted_values = []
    
    for sublist in predictions:
        
        for item in sublist:
            predicted_values.append(item)

    tab_predict = []

    for i in range(3):
        tab_predict.append(predicted_values[i])

    max_index = tab_predict.index(max(tab_predict))

    print("\n", tab_predict, "\n")
    print(CLASSES[max_index])

# III) D) Implémentation d'un PMC avec deux couches cachées à 32 neurones

In [9]:
def train_mlp_tl_ttn_model():
    
    mlp_tl_ttn_model = keras.models.Sequential()
    mlp_tl_ttn_model.add(keras.layers.Input(shape=x_train.shape[1:]))
    mlp_tl_ttn_model.add(keras.layers.Dense(32, activation=keras.activations.relu))
    mlp_tl_ttn_model.add(keras.layers.Dense(32, activation=keras.activations.relu))
    mlp_tl_ttn_model.add(keras.layers.Dense(3, activation=keras.activations.softmax))

    mlp_tl_ttn_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss=keras.losses.CategoricalCrossentropy(from_logits=False), metrics=keras.metrics.CategoricalAccuracy(name="accuracy"))
    
    print(mlp_tl_ttn_model.summary())
    
    predicted_values_before_train = mlp_tl_ttn_model.predict(x_train)

    model_name = 'mlp_tl_ttn_model'
    log_dir_path = './logs'

    if not os.path.exists(log_dir_path):
        os.mkdir(log_dir_path)

    full_log_path = os.path.join(log_dir_path, model_name)

    if not os.path.exists(full_log_path):
        os.mkdir(full_log_path)

    mlp_tl_ttn_model.fit(x_train, y_train, epochs=1000, validation_data=(x_test, y_test), callbacks=[

        keras.callbacks.TensorBoard(full_log_path, )

    ])

    print("Valeurs de prédiction avant entrainement : \n", predicted_values_before_train)
    print("Valeurs de prédiction après entrainement : \n", mlp_tl_ttn_model.predict(x_train))
    
    mlp_tl_ttn_model.save("../saves/mlp_tl_ttn_model.h5")

In [10]:
def predict_mlp_tl_ttn_model():
    
    loaded_mlp_tl_ttn_model = keras.models.load_model("../saves/mlp_tl_ttn_model.h5")
    
    image = Image.open(test_file)
    if image.mode in ("RGB", "P"): image = image.convert("RGBA")

    image = image.resize((32, 32))

    im_arr = np.array(image).flatten()
    im_arr = im_arr / 255.0
    im_arr = im_arr.tolist()
    im_arr = np.array(im_arr)
    im_arr = np.expand_dims(im_arr, axis=0)
    
    predict = loaded_mlp_tl_ttn_model.predict(im_arr)
    predictions = predict.tolist()
    
    predicted_values = []
    
    for sublist in predictions:
        
        for item in sublist:
            predicted_values.append(item)

    tab_predict = []

    for i in range(3):
        tab_predict.append(predicted_values[i])

    max_index = tab_predict.index(max(tab_predict))

    print("\n", tab_predict, "\n")
    print(CLASSES[max_index])

# IV) Exécution des modèles et prédiction des résultats

# IV) A) Exécution d'un perceptron simple

In [11]:
if __name__ == "__main__":
    
    train_mlp_wl_model()
    predict_mlp_wl_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12291     
Total params: 12,291
Trainable params: 12,291
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
19/19 [==============================] - 1s 38ms/step - loss: 1.1456 - accuracy: 0.3317 - val_loss: 1.1250 - val_accuracy: 0.3917
Epoch 2/1000
19/19 [==============================] - 0s 4ms/step - loss: 1.0451 - accuracy: 0.4533 - val_loss: 0.9672 - val_accuracy: 0.6583
Epoch 3/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.9516 - accuracy: 0.5800 - val_loss: 0.9143 - val_accuracy: 0.6333
Epoch 4/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.9156 - accuracy: 0.6100 - val_loss: 0.8514 - val_accuracy: 0.7000
Epoch 5/1000
19/19 [==============================] - 0s 4ms/step - loss

Epoch 54/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5554 - accuracy: 0.7900 - val_loss: 0.6014 - val_accuracy: 0.7000
Epoch 55/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5481 - accuracy: 0.7967 - val_loss: 0.6134 - val_accuracy: 0.7333
Epoch 56/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5594 - accuracy: 0.7800 - val_loss: 0.5871 - val_accuracy: 0.7667
Epoch 57/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5475 - accuracy: 0.7833 - val_loss: 0.5909 - val_accuracy: 0.7417
Epoch 58/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5484 - accuracy: 0.7817 - val_loss: 0.5971 - val_accuracy: 0.6917
Epoch 59/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5417 - accuracy: 0.7983 - val_loss: 0.5932 - val_accuracy: 0.7583
Epoch 60/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5456 - accuracy: 0.7817 - val_loss: 0.5928 - val_accuracy: 0.7417

Epoch 111/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.4868 - accuracy: 0.8100 - val_loss: 0.5713 - val_accuracy: 0.7417
Epoch 112/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4807 - accuracy: 0.8117 - val_loss: 0.5655 - val_accuracy: 0.7417
Epoch 113/1000
19/19 [==============================] - 0s 5ms/step - loss: 0.4755 - accuracy: 0.8217 - val_loss: 0.5648 - val_accuracy: 0.7083
Epoch 114/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4809 - accuracy: 0.8167 - val_loss: 0.5612 - val_accuracy: 0.7333
Epoch 115/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.4780 - accuracy: 0.8300 - val_loss: 0.5800 - val_accuracy: 0.7500
Epoch 116/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.4840 - accuracy: 0.8133 - val_loss: 0.5621 - val_accuracy: 0.7417
Epoch 117/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.4763 - accuracy: 0.8183 - val_loss: 0.5708 - val_accuracy:

Epoch 224/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4094 - accuracy: 0.8500 - val_loss: 0.5555 - val_accuracy: 0.7500
Epoch 225/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4165 - accuracy: 0.8500 - val_loss: 0.5633 - val_accuracy: 0.7417
Epoch 226/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4076 - accuracy: 0.8550 - val_loss: 0.5856 - val_accuracy: 0.7083
Epoch 227/1000
19/19 [==============================] - ETA: 0s - loss: 0.2948 - accuracy: 0.93 - 0s 4ms/step - loss: 0.4084 - accuracy: 0.8567 - val_loss: 0.5468 - val_accuracy: 0.7417
Epoch 228/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4077 - accuracy: 0.8633 - val_loss: 0.5550 - val_accuracy: 0.7250
Epoch 229/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4065 - accuracy: 0.8533 - val_loss: 0.5505 - val_accuracy: 0.7250
Epoch 230/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.4070 - accuracy:

Epoch 337/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.3658 - accuracy: 0.8767 - val_loss: 0.5359 - val_accuracy: 0.7167
Epoch 338/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3638 - accuracy: 0.8783 - val_loss: 0.5432 - val_accuracy: 0.7417
Epoch 339/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.3644 - accuracy: 0.8750 - val_loss: 0.5385 - val_accuracy: 0.7417
Epoch 340/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.3633 - accuracy: 0.8833 - val_loss: 0.5404 - val_accuracy: 0.7417
Epoch 341/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.3619 - accuracy: 0.8733 - val_loss: 0.5613 - val_accuracy: 0.7417
Epoch 342/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3625 - accuracy: 0.8767 - val_loss: 0.5363 - val_accuracy: 0.7167
Epoch 343/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3604 - accuracy: 0.8850 - val_loss: 0.5353 - val_accuracy:

...

Epoch 565/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3062 - accuracy: 0.9000 - val_loss: 0.5587 - val_accuracy: 0.7250
Epoch 566/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3058 - accuracy: 0.9033 - val_loss: 0.5343 - val_accuracy: 0.7500
Epoch 567/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3032 - accuracy: 0.8967 - val_loss: 0.5377 - val_accuracy: 0.7417
Epoch 568/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3066 - accuracy: 0.9067 - val_loss: 0.5283 - val_accuracy: 0.7417
Epoch 569/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3080 - accuracy: 0.9117 - val_loss: 0.5394 - val_accuracy: 0.7500
Epoch 570/1000
19/19 [==============================] - 0s 5ms/step - loss: 0.3038 - accuracy: 0.9083 - val_loss: 0.5298 - val_accuracy: 0.7500
Epoch 571/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3049 - accuracy: 0.9050 - val_loss: 0.5463 - val_accuracy:

Epoch 678/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2849 - accuracy: 0.9083 - val_loss: 0.5442 - val_accuracy: 0.7417
Epoch 679/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2871 - accuracy: 0.9100 - val_loss: 0.5508 - val_accuracy: 0.7333
Epoch 680/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2851 - accuracy: 0.9100 - val_loss: 0.5553 - val_accuracy: 0.7333
Epoch 681/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2847 - accuracy: 0.9133 - val_loss: 0.5523 - val_accuracy: 0.7417
Epoch 682/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2855 - accuracy: 0.9133 - val_loss: 0.5699 - val_accuracy: 0.7083
Epoch 683/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2881 - accuracy: 0.9083 - val_loss: 0.5395 - val_accuracy: 0.7417
Epoch 684/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2854 - accuracy: 0.9133 - val_loss: 0.5294 - val_accuracy:

Epoch 735/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2770 - accuracy: 0.9167 - val_loss: 0.5523 - val_accuracy: 0.7250
Epoch 736/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2791 - accuracy: 0.9217 - val_loss: 0.5644 - val_accuracy: 0.7167
Epoch 737/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2757 - accuracy: 0.9183 - val_loss: 0.5522 - val_accuracy: 0.7500
Epoch 738/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2749 - accuracy: 0.9167 - val_loss: 0.5391 - val_accuracy: 0.7333
Epoch 739/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2755 - accuracy: 0.9083 - val_loss: 0.5396 - val_accuracy: 0.7500
Epoch 740/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2762 - accuracy: 0.9183 - val_loss: 0.5488 - val_accuracy: 0.7500
Epoch 741/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2748 - accuracy: 0.9150 - val_loss: 0.5490 - val_accuracy:

Epoch 848/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2601 - accuracy: 0.9250 - val_loss: 0.5555 - val_accuracy: 0.7333
Epoch 849/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2604 - accuracy: 0.9233 - val_loss: 0.5400 - val_accuracy: 0.7417
Epoch 850/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2623 - accuracy: 0.9233 - val_loss: 0.5452 - val_accuracy: 0.7417
Epoch 851/1000
19/19 [==============================] - 0s 8ms/step - loss: 0.2582 - accuracy: 0.9283 - val_loss: 0.5580 - val_accuracy: 0.7500
Epoch 852/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2612 - accuracy: 0.9233 - val_loss: 0.5510 - val_accuracy: 0.7583
Epoch 853/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2606 - accuracy: 0.9300 - val_loss: 0.5533 - val_accuracy: 0.7500
Epoch 854/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2607 - accuracy: 0.9200 - val_loss: 0.5414 - val_accuracy:

Epoch 905/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2523 - accuracy: 0.9317 - val_loss: 0.5417 - val_accuracy: 0.7333
Epoch 906/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2540 - accuracy: 0.9217 - val_loss: 0.5450 - val_accuracy: 0.7417
Epoch 907/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2535 - accuracy: 0.9300 - val_loss: 0.5337 - val_accuracy: 0.7667
Epoch 908/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.2562 - accuracy: 0.9267 - val_loss: 0.5368 - val_accuracy: 0.7583
Epoch 909/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2543 - accuracy: 0.9317 - val_loss: 0.5420 - val_accuracy: 0.7417
Epoch 910/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2516 - accuracy: 0.9350 - val_loss: 0.5681 - val_accuracy: 0.7250
Epoch 911/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2521 - accuracy: 0.9350 - val_loss: 0.5595 - val_accuracy:

# IV) B) Exécution d'un PCM avec une couche cachée à 8 neurones

In [12]:
train_mlp_ol_en_model()
predict_mlp_ol_en_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 32776     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 27        
Total params: 32,803
Trainable params: 32,803
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
19/19 [==============================] - 1s 52ms/step - loss: 1.2449 - accuracy: 0.2767 - val_loss: 1.1285 - val_accuracy: 0.3417
Epoch 2/1000
19/19 [==============================] - 0s 4ms/step - loss: 1.0698 - accuracy: 0.4233 - val_loss: 1.0023 - val_accuracy: 0.5333
Epoch 3/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.9712 - accuracy: 0.5350 - val_loss: 0.9238 - val_accuracy: 0.5750
Epoch 4/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.9368

19/19 [==============================] - 0s 4ms/step - loss: 0.4509 - accuracy: 0.8283 - val_loss: 0.6046 - val_accuracy: 0.7500
Epoch 110/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4644 - accuracy: 0.8233 - val_loss: 0.5874 - val_accuracy: 0.7333
Epoch 111/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4520 - accuracy: 0.8250 - val_loss: 0.5841 - val_accuracy: 0.7167
Epoch 112/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4558 - accuracy: 0.8167 - val_loss: 0.5707 - val_accuracy: 0.7417
Epoch 113/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4503 - accuracy: 0.8267 - val_loss: 0.5819 - val_accuracy: 0.7333
Epoch 114/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.4470 - accuracy: 0.8233 - val_loss: 0.5886 - val_accuracy: 0.7083
Epoch 115/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4476 - accuracy: 0.8250 - val_loss: 0.5770 - val_accuracy: 0.7083
Epoch 1

Epoch 166/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4001 - accuracy: 0.8433 - val_loss: 0.5907 - val_accuracy: 0.7167
Epoch 167/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3953 - accuracy: 0.8533 - val_loss: 0.5818 - val_accuracy: 0.7167
Epoch 168/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3968 - accuracy: 0.8517 - val_loss: 0.5655 - val_accuracy: 0.7500
Epoch 169/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3917 - accuracy: 0.8617 - val_loss: 0.6049 - val_accuracy: 0.7167
Epoch 170/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4024 - accuracy: 0.8450 - val_loss: 0.5741 - val_accuracy: 0.6917
Epoch 171/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3974 - accuracy: 0.8517 - val_loss: 0.5817 - val_accuracy: 0.7333
Epoch 172/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3877 - accuracy: 0.8567 - val_loss: 0.5709 - val_accuracy:

Epoch 223/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3564 - accuracy: 0.8700 - val_loss: 0.5565 - val_accuracy: 0.7417
Epoch 224/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3496 - accuracy: 0.8883 - val_loss: 0.5658 - val_accuracy: 0.7167
Epoch 225/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3580 - accuracy: 0.8650 - val_loss: 0.5816 - val_accuracy: 0.7167
Epoch 226/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.3612 - accuracy: 0.8633 - val_loss: 0.5809 - val_accuracy: 0.7167
Epoch 227/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3503 - accuracy: 0.8750 - val_loss: 0.5688 - val_accuracy: 0.7167
Epoch 228/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3495 - accuracy: 0.8783 - val_loss: 0.5701 - val_accuracy: 0.7167
Epoch 229/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3475 - accuracy: 0.8783 - val_loss: 0.5534 - val_accuracy:

Epoch 280/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3180 - accuracy: 0.8950 - val_loss: 0.5932 - val_accuracy: 0.7083
Epoch 281/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3137 - accuracy: 0.8900 - val_loss: 0.5527 - val_accuracy: 0.7333
Epoch 282/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3134 - accuracy: 0.8983 - val_loss: 0.5516 - val_accuracy: 0.7667
Epoch 283/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3264 - accuracy: 0.8933 - val_loss: 0.5860 - val_accuracy: 0.7250
Epoch 284/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3150 - accuracy: 0.8983 - val_loss: 0.5684 - val_accuracy: 0.7167
Epoch 285/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3104 - accuracy: 0.9017 - val_loss: 0.5813 - val_accuracy: 0.7000
Epoch 286/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3106 - accuracy: 0.9000 - val_loss: 0.5794 - val_accuracy:

...

Epoch 676/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1710 - accuracy: 0.9533 - val_loss: 0.6319 - val_accuracy: 0.7333
Epoch 677/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1718 - accuracy: 0.9617 - val_loss: 0.6585 - val_accuracy: 0.7333
Epoch 678/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1732 - accuracy: 0.9600 - val_loss: 0.6217 - val_accuracy: 0.7417
Epoch 679/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1742 - accuracy: 0.9600 - val_loss: 0.6742 - val_accuracy: 0.7000
Epoch 680/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1701 - accuracy: 0.9650 - val_loss: 0.6432 - val_accuracy: 0.7333
Epoch 681/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1652 - accuracy: 0.9683 - val_loss: 0.6512 - val_accuracy: 0.7417
Epoch 682/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1730 - accuracy: 0.9633 - val_loss: 0.6318 - val_accuracy:

Epoch 733/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1668 - accuracy: 0.9633 - val_loss: 0.6310 - val_accuracy: 0.7333
Epoch 734/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.1574 - accuracy: 0.9650 - val_loss: 0.6570 - val_accuracy: 0.7167
Epoch 735/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1603 - accuracy: 0.9733 - val_loss: 0.6054 - val_accuracy: 0.7500
Epoch 736/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.1605 - accuracy: 0.9600 - val_loss: 0.6084 - val_accuracy: 0.7667
Epoch 737/1000
19/19 [==============================] - 0s 9ms/step - loss: 0.1580 - accuracy: 0.9667 - val_loss: 0.6278 - val_accuracy: 0.7167
Epoch 738/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1544 - accuracy: 0.9683 - val_loss: 0.6090 - val_accuracy: 0.7333
Epoch 739/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1560 - accuracy: 0.9667 - val_loss: 0.6102 - val_accuracy:

Epoch 846/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1339 - accuracy: 0.9783 - val_loss: 0.6567 - val_accuracy: 0.7500
Epoch 847/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1338 - accuracy: 0.9750 - val_loss: 0.6218 - val_accuracy: 0.7500
Epoch 848/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1315 - accuracy: 0.9800 - val_loss: 0.6662 - val_accuracy: 0.7500
Epoch 849/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1420 - accuracy: 0.9783 - val_loss: 0.6629 - val_accuracy: 0.7333
Epoch 850/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1348 - accuracy: 0.9783 - val_loss: 0.6388 - val_accuracy: 0.7417
Epoch 851/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1345 - accuracy: 0.9767 - val_loss: 0.7057 - val_accuracy: 0.7333
Epoch 852/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1313 - accuracy: 0.9817 - val_loss: 0.6397 - val_accuracy:

Epoch 903/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1242 - accuracy: 0.9767 - val_loss: 0.6703 - val_accuracy: 0.7500
Epoch 904/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1230 - accuracy: 0.9833 - val_loss: 0.6919 - val_accuracy: 0.7250
Epoch 905/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.1238 - accuracy: 0.9850 - val_loss: 0.6533 - val_accuracy: 0.7500
Epoch 906/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.1246 - accuracy: 0.9817 - val_loss: 0.6935 - val_accuracy: 0.7250
Epoch 907/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.1264 - accuracy: 0.9800 - val_loss: 0.6380 - val_accuracy: 0.7417
Epoch 908/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.1234 - accuracy: 0.9867 - val_loss: 0.6864 - val_accuracy: 0.7333
Epoch 909/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.1242 - accuracy: 0.9800 - val_loss: 0.6818 - val_accuracy:

Epoch 960/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1174 - accuracy: 0.9817 - val_loss: 0.6615 - val_accuracy: 0.7417
Epoch 961/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1163 - accuracy: 0.9833 - val_loss: 0.6820 - val_accuracy: 0.7667
Epoch 962/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1205 - accuracy: 0.9817 - val_loss: 0.6874 - val_accuracy: 0.7333
Epoch 963/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1140 - accuracy: 0.9800 - val_loss: 0.6787 - val_accuracy: 0.7417
Epoch 964/1000
19/19 [==============================] - 0s 6ms/step - loss: 0.1167 - accuracy: 0.9817 - val_loss: 0.6763 - val_accuracy: 0.7500
Epoch 965/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1131 - accuracy: 0.9883 - val_loss: 0.6748 - val_accuracy: 0.7500
Epoch 966/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1129 - accuracy: 0.9900 - val_loss: 0.6666 - val_accuracy:

# IV) C) Exécution d'un PCM avec une couche cachée à 32 neurones

In [13]:
train_mlp_ol_ttn_model()
predict_mlp_ol_ttn_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                131104    
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 99        
Total params: 131,203
Trainable params: 131,203
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
19/19 [==============================] - 1s 48ms/step - loss: 1.0383 - accuracy: 0.4550 - val_loss: 0.9967 - val_accuracy: 0.5250
Epoch 2/1000
19/19 [==============================] - 0s 5ms/step - loss: 0.9540 - accuracy: 0.5983 - val_loss: 0.9344 - val_accuracy: 0.5417
Epoch 3/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.9064 - accuracy: 0.6133 - val_loss: 0.8840 - val_accuracy: 0.5833
Epoch 4/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.86

19/19 [==============================] - 0s 4ms/step - loss: 0.4627 - accuracy: 0.8150 - val_loss: 0.5618 - val_accuracy: 0.7583
Epoch 110/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4593 - accuracy: 0.8183 - val_loss: 0.5418 - val_accuracy: 0.7667
Epoch 111/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4610 - accuracy: 0.8200 - val_loss: 0.6400 - val_accuracy: 0.7167
Epoch 112/1000
19/19 [==============================] - 0s 5ms/step - loss: 0.4675 - accuracy: 0.8133 - val_loss: 0.5416 - val_accuracy: 0.7583
Epoch 113/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4597 - accuracy: 0.8333 - val_loss: 0.5676 - val_accuracy: 0.7417
Epoch 114/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4588 - accuracy: 0.8183 - val_loss: 0.5421 - val_accuracy: 0.7500
Epoch 115/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4593 - accuracy: 0.8217 - val_loss: 0.5550 - val_accuracy: 0.7417
Epoch 1

Epoch 166/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4042 - accuracy: 0.8517 - val_loss: 0.5633 - val_accuracy: 0.7417
Epoch 167/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4038 - accuracy: 0.8567 - val_loss: 0.5379 - val_accuracy: 0.7583
Epoch 168/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4079 - accuracy: 0.8583 - val_loss: 0.5545 - val_accuracy: 0.7250
Epoch 169/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4006 - accuracy: 0.8417 - val_loss: 0.5377 - val_accuracy: 0.7583
Epoch 170/1000
19/19 [==============================] - 0s 5ms/step - loss: 0.3995 - accuracy: 0.8533 - val_loss: 0.5303 - val_accuracy: 0.7417
Epoch 171/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3940 - accuracy: 0.8550 - val_loss: 0.5375 - val_accuracy: 0.7500
Epoch 172/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4001 - accuracy: 0.8500 - val_loss: 0.5458 - val_accuracy:

Epoch 223/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3681 - accuracy: 0.8633 - val_loss: 0.5431 - val_accuracy: 0.7417
Epoch 224/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3603 - accuracy: 0.8583 - val_loss: 0.5466 - val_accuracy: 0.7250
Epoch 225/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3643 - accuracy: 0.8750 - val_loss: 0.5309 - val_accuracy: 0.7417
Epoch 226/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3554 - accuracy: 0.8767 - val_loss: 0.5555 - val_accuracy: 0.7417
Epoch 227/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3536 - accuracy: 0.8783 - val_loss: 0.5648 - val_accuracy: 0.7333
Epoch 228/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3610 - accuracy: 0.8683 - val_loss: 0.5288 - val_accuracy: 0.7417
Epoch 229/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3589 - accuracy: 0.8633 - val_loss: 0.5408 - val_accuracy:

Epoch 280/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3235 - accuracy: 0.8783 - val_loss: 0.5392 - val_accuracy: 0.7250
Epoch 281/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3204 - accuracy: 0.8900 - val_loss: 0.5228 - val_accuracy: 0.7583
Epoch 282/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3329 - accuracy: 0.8867 - val_loss: 0.5373 - val_accuracy: 0.7333
Epoch 283/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3252 - accuracy: 0.8933 - val_loss: 0.5573 - val_accuracy: 0.7167
Epoch 284/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3238 - accuracy: 0.8917 - val_loss: 0.5225 - val_accuracy: 0.7333
Epoch 285/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3198 - accuracy: 0.8800 - val_loss: 0.5533 - val_accuracy: 0.7167
Epoch 286/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3226 - accuracy: 0.8917 - val_loss: 0.5232 - val_accuracy:

...

Epoch 676/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1708 - accuracy: 0.9633 - val_loss: 0.5633 - val_accuracy: 0.7500
Epoch 677/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1680 - accuracy: 0.9633 - val_loss: 0.5817 - val_accuracy: 0.7000
Epoch 678/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1697 - accuracy: 0.9600 - val_loss: 0.5536 - val_accuracy: 0.7750
Epoch 679/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1691 - accuracy: 0.9600 - val_loss: 0.5533 - val_accuracy: 0.7667
Epoch 680/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1678 - accuracy: 0.9617 - val_loss: 0.5948 - val_accuracy: 0.7250
Epoch 681/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1677 - accuracy: 0.9633 - val_loss: 0.5680 - val_accuracy: 0.7667
Epoch 682/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1667 - accuracy: 0.9600 - val_loss: 0.5784 - val_accuracy:

Epoch 733/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1566 - accuracy: 0.9650 - val_loss: 0.6481 - val_accuracy: 0.7417
Epoch 734/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1577 - accuracy: 0.9633 - val_loss: 0.5969 - val_accuracy: 0.7333
Epoch 735/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1544 - accuracy: 0.9717 - val_loss: 0.5941 - val_accuracy: 0.7667
Epoch 736/1000
19/19 [==============================] - 0s 5ms/step - loss: 0.1525 - accuracy: 0.9733 - val_loss: 0.5778 - val_accuracy: 0.7167
Epoch 737/1000
19/19 [==============================] - 0s 7ms/step - loss: 0.1551 - accuracy: 0.9683 - val_loss: 0.5951 - val_accuracy: 0.7333
Epoch 738/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1521 - accuracy: 0.9700 - val_loss: 0.6303 - val_accuracy: 0.7250
Epoch 739/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1545 - accuracy: 0.9667 - val_loss: 0.5968 - val_accuracy:

Epoch 846/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1283 - accuracy: 0.9767 - val_loss: 0.6143 - val_accuracy: 0.7333
Epoch 847/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.1267 - accuracy: 0.9833 - val_loss: 0.6097 - val_accuracy: 0.7667
Epoch 848/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1289 - accuracy: 0.9817 - val_loss: 0.6225 - val_accuracy: 0.7083
Epoch 849/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1282 - accuracy: 0.9867 - val_loss: 0.6361 - val_accuracy: 0.7083
Epoch 850/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1273 - accuracy: 0.9783 - val_loss: 0.6328 - val_accuracy: 0.7417
Epoch 851/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1284 - accuracy: 0.9833 - val_loss: 0.6004 - val_accuracy: 0.7500
Epoch 852/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1293 - accuracy: 0.9833 - val_loss: 0.6086 - val_accuracy:

Epoch 903/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1192 - accuracy: 0.9833 - val_loss: 0.6137 - val_accuracy: 0.7333
Epoch 904/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.1192 - accuracy: 0.9867 - val_loss: 0.6175 - val_accuracy: 0.7333
Epoch 905/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1206 - accuracy: 0.9867 - val_loss: 0.6337 - val_accuracy: 0.7250
Epoch 906/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1215 - accuracy: 0.9800 - val_loss: 0.6014 - val_accuracy: 0.7500
Epoch 907/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1198 - accuracy: 0.9867 - val_loss: 0.6837 - val_accuracy: 0.7333
Epoch 908/1000
19/19 [==============================] - 0s 5ms/step - loss: 0.1201 - accuracy: 0.9833 - val_loss: 0.6250 - val_accuracy: 0.7500
Epoch 909/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1181 - accuracy: 0.9833 - val_loss: 0.6138 - val_accuracy:

Epoch 960/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1099 - accuracy: 0.9883 - val_loss: 0.6378 - val_accuracy: 0.7083
Epoch 961/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1083 - accuracy: 0.9917 - val_loss: 0.6256 - val_accuracy: 0.7500
Epoch 962/1000
19/19 [==============================] - 0s 5ms/step - loss: 0.1072 - accuracy: 0.9917 - val_loss: 0.6073 - val_accuracy: 0.7583
Epoch 963/1000
19/19 [==============================] - 0s 8ms/step - loss: 0.1114 - accuracy: 0.9850 - val_loss: 0.6236 - val_accuracy: 0.7500
Epoch 964/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1084 - accuracy: 0.9917 - val_loss: 0.6143 - val_accuracy: 0.7583
Epoch 965/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1118 - accuracy: 0.9833 - val_loss: 0.6731 - val_accuracy: 0.7167
Epoch 966/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.1062 - accuracy: 0.9883 - val_loss: 0.6224 - val_accuracy:

# IV) D) Exécution d'un PCM avec deux couches cachées à 32 neurones

In [14]:
train_mlp_tl_ttn_model()
predict_mlp_tl_ttn_model()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 32)                131104    
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 99        
Total params: 132,259
Trainable params: 132,259
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
19/19 [==============================] - 1s 50ms/step - loss: 1.1531 - accuracy: 0.3233 - val_loss: 1.0096 - val_accuracy: 0.5333
Epoch 2/1000
19/19 [==============================] - 0s 5ms/step - loss: 1.0297 - accuracy: 0.4833 - val_loss: 0.9436 - val_accuracy: 0.5583
Epoch 3/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.9795 - accur

Epoch 52/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5242 - accuracy: 0.7850 - val_loss: 0.5860 - val_accuracy: 0.7167
Epoch 53/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5236 - accuracy: 0.7800 - val_loss: 0.5879 - val_accuracy: 0.7417
Epoch 54/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5265 - accuracy: 0.7833 - val_loss: 0.5856 - val_accuracy: 0.7500
Epoch 55/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5116 - accuracy: 0.7900 - val_loss: 0.6102 - val_accuracy: 0.7333
Epoch 56/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5215 - accuracy: 0.7800 - val_loss: 0.5649 - val_accuracy: 0.7500
Epoch 57/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5071 - accuracy: 0.7983 - val_loss: 0.5683 - val_accuracy: 0.7250
Epoch 58/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.5180 - accuracy: 0.7633 - val_loss: 0.5972 - val_accuracy: 0.7500

Epoch 109/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4125 - accuracy: 0.8383 - val_loss: 0.5444 - val_accuracy: 0.7583
Epoch 110/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4227 - accuracy: 0.8283 - val_loss: 0.5420 - val_accuracy: 0.7500
Epoch 111/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4084 - accuracy: 0.8350 - val_loss: 0.5528 - val_accuracy: 0.7333
Epoch 112/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4169 - accuracy: 0.8350 - val_loss: 0.6146 - val_accuracy: 0.7333
Epoch 113/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4067 - accuracy: 0.8333 - val_loss: 0.5212 - val_accuracy: 0.7583
Epoch 114/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4106 - accuracy: 0.8383 - val_loss: 0.5314 - val_accuracy: 0.7500
Epoch 115/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.4026 - accuracy: 0.8367 - val_loss: 0.5625 - val_accuracy:

Epoch 166/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3456 - accuracy: 0.8733 - val_loss: 0.5153 - val_accuracy: 0.7583
Epoch 167/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3519 - accuracy: 0.8617 - val_loss: 0.5080 - val_accuracy: 0.7667
Epoch 168/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3458 - accuracy: 0.8683 - val_loss: 0.5194 - val_accuracy: 0.7583
Epoch 169/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3494 - accuracy: 0.8783 - val_loss: 0.6575 - val_accuracy: 0.7250
Epoch 170/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3586 - accuracy: 0.8683 - val_loss: 0.5554 - val_accuracy: 0.7667
Epoch 171/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3435 - accuracy: 0.8833 - val_loss: 0.5830 - val_accuracy: 0.7500
Epoch 172/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3532 - accuracy: 0.8750 - val_loss: 0.6203 - val_accuracy:

Epoch 223/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.3008 - accuracy: 0.8967 - val_loss: 0.5449 - val_accuracy: 0.7583
Epoch 224/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2948 - accuracy: 0.9017 - val_loss: 0.5250 - val_accuracy: 0.7417
Epoch 225/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2898 - accuracy: 0.9033 - val_loss: 0.5053 - val_accuracy: 0.7417
Epoch 226/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2989 - accuracy: 0.8983 - val_loss: 0.5246 - val_accuracy: 0.7333
Epoch 227/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2933 - accuracy: 0.8967 - val_loss: 0.5156 - val_accuracy: 0.7583
Epoch 228/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2932 - accuracy: 0.8917 - val_loss: 0.5239 - val_accuracy: 0.7333
Epoch 229/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.2980 - accuracy: 0.8883 - val_loss: 0.5104 - val_accuracy:

...

Epoch 733/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.0730 - accuracy: 0.9950 - val_loss: 0.7070 - val_accuracy: 0.7333
Epoch 734/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0723 - accuracy: 0.9950 - val_loss: 0.7282 - val_accuracy: 0.7500
Epoch 735/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.0686 - accuracy: 0.9967 - val_loss: 0.7434 - val_accuracy: 0.7417
Epoch 736/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0681 - accuracy: 0.9950 - val_loss: 0.6837 - val_accuracy: 0.7250
Epoch 737/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0681 - accuracy: 0.9917 - val_loss: 0.6948 - val_accuracy: 0.7333
Epoch 738/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0688 - accuracy: 0.9950 - val_loss: 0.6948 - val_accuracy: 0.7333
Epoch 739/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0723 - accuracy: 0.9917 - val_loss: 0.7071 - val_accuracy:

Epoch 790/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0575 - accuracy: 0.9950 - val_loss: 0.7462 - val_accuracy: 0.7417
Epoch 791/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0578 - accuracy: 0.9967 - val_loss: 0.7175 - val_accuracy: 0.7333
Epoch 792/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0602 - accuracy: 0.9933 - val_loss: 0.7668 - val_accuracy: 0.7250
Epoch 793/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0615 - accuracy: 0.9917 - val_loss: 0.7059 - val_accuracy: 0.7417
Epoch 794/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0584 - accuracy: 0.9917 - val_loss: 0.7399 - val_accuracy: 0.7417
Epoch 795/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0590 - accuracy: 0.9933 - val_loss: 0.7464 - val_accuracy: 0.7583
Epoch 796/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0566 - accuracy: 0.9967 - val_loss: 0.7006 - val_accuracy:

Epoch 847/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0540 - accuracy: 0.9967 - val_loss: 0.7816 - val_accuracy: 0.7500
Epoch 848/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0515 - accuracy: 0.9950 - val_loss: 0.7336 - val_accuracy: 0.7333
Epoch 849/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0513 - accuracy: 0.9950 - val_loss: 0.7357 - val_accuracy: 0.7417
Epoch 850/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0496 - accuracy: 0.9950 - val_loss: 0.7705 - val_accuracy: 0.7500
Epoch 851/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0490 - accuracy: 0.9983 - val_loss: 0.7872 - val_accuracy: 0.7417
Epoch 852/1000
19/19 [==============================] - 0s 2ms/step - loss: 0.0516 - accuracy: 0.9950 - val_loss: 0.7411 - val_accuracy: 0.7333
Epoch 853/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0527 - accuracy: 0.9967 - val_loss: 0.7513 - val_accuracy:

Epoch 904/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0426 - accuracy: 0.9983 - val_loss: 0.7630 - val_accuracy: 0.7250
Epoch 905/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.0430 - accuracy: 1.0000 - val_loss: 0.7482 - val_accuracy: 0.7250
Epoch 906/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0438 - accuracy: 0.9983 - val_loss: 0.7954 - val_accuracy: 0.7333
Epoch 907/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0440 - accuracy: 0.9967 - val_loss: 0.7899 - val_accuracy: 0.7250
Epoch 908/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0462 - accuracy: 0.9983 - val_loss: 0.7922 - val_accuracy: 0.7500
Epoch 909/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.0427 - accuracy: 0.9983 - val_loss: 0.7578 - val_accuracy: 0.7333
Epoch 910/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0418 - accuracy: 0.9983 - val_loss: 0.7469 - val_accuracy:

Epoch 961/1000
19/19 [==============================] - 0s 4ms/step - loss: 0.0378 - accuracy: 0.9983 - val_loss: 0.7962 - val_accuracy: 0.7500
Epoch 962/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0373 - accuracy: 0.9983 - val_loss: 0.7958 - val_accuracy: 0.7333
Epoch 963/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0380 - accuracy: 0.9983 - val_loss: 0.7728 - val_accuracy: 0.7250
Epoch 964/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0370 - accuracy: 1.0000 - val_loss: 0.7629 - val_accuracy: 0.7333
Epoch 965/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0387 - accuracy: 0.9983 - val_loss: 0.8015 - val_accuracy: 0.7500
Epoch 966/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0391 - accuracy: 0.9983 - val_loss: 0.8250 - val_accuracy: 0.7417
Epoch 967/1000
19/19 [==============================] - 0s 3ms/step - loss: 0.0383 - accuracy: 0.9983 - val_loss: 0.8012 - val_accuracy: